## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [32]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[720, 721, 722]


In [33]:
from IPython.utils.text import indent
import requests
import json

sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 720,
    "name": "Pytherson",
    "account": {
      "id": 744,
      "number": "00025-1",
      "agency": "string",
      "balance": 0.0,
      "limit": 0.0
    },
    "card": {
      "id": 695,
      "number": "**** **** **** 2050",
      "limit": 1000.0
    },
    "features": [],
    "news": [
      {
        "id": 1985,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Invista no seu futuro, seja Pytherson!"
      },
      {
        "id": 1986,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Invista hoje, conquiste amanh\u00e3!"
      },
      {
        "id": 1987,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Investir \u00e9 o caminho para um futuro financeiro est\u00e1vel."
      },
      {
        "id": 1988,
        "icon": "http

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [42]:
!pip install openai

In [43]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'API_KEY'


In [44]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
        "role": "system",
        "content": "Você é um especialista em marketing bancário."
      },
      {
        "role": "user",
        "content": f"Crie uma mensagem para {user['name']} sobre a importancia dos investimentos (máximo 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Invista hoje, colha amanhã!
Invista no seu futuro financeiro!
Invista hoje, colha amanhã!


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [45]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}")

User Pytherson updated? True
User Louis updated? True
User Paul updated? True
